# What does this program do?

IAA is a optimizer tool -- its true meaning is (I)nteractive (A)sset (A)llocator. The intention is to aid the user in portfolio development with assets of their own choice. 

IAA reads from the standard input: it takes in the number of assets to be invested, followed by the previous day's date & a list of tickers the user wishes to purchase. For example, today is May 25th, 2024, so a valid input example would be:

5 2024-05-24 AAPL AMZN MSFT GOGL META

or: 8 2024-05-24 WMT NFLX IBM W SBUX BKC KRW SHOP

The IAA only accepts valid tickers and stocks denominated in CAD/USD. It will reject anything that does not follow this. Its ultimate goal is to return a distribution of asset allocation that optimizes for the greatest possible return. 

Keep in mind that it will not necessarily produce a portfolio that is highly profitable. 

Instead, it provides the user with the MOST profitable portfolio distribution given assets THEY choose.


In [1]:
import pandas as pd
import numpy as np 
import yfinance as yf
import datetime as dt
from scipy.optimize import minimize

# importing necessary modules and creating aliases (1)

In [2]:
num_assets = input("How many assets are you interested in managing? (must be an integer): ")
num_assets = int(num_assets)
date = input("Enter yesterday's date (format should be yyyy-mm-dd): ")
trading_days = 252

How many assets are you interested in managing? (must be an integer): 5
Enter yesterday's date (format should be yyyy-mm-dd): 2024-05-24


In [3]:
def valid_curr(ticker):
    stock_info = yf.Ticker(ticker).info
    currency = stock_info.get('currency', '').upper() # retrieving the stock currency
    return currency == 'USD' or currency == 'CAD' # returns T/F based on currency

In [4]:
assets = []
asset_str = ""

while len(assets) < num_assets:
    asset = input("What is the ticker of your desired stock?: ").upper()
    if valid_curr(asset):
        assets.append(asset)
    else:
        print("\n")
        print("The ticker you have entered is invalid or is not denominated in USD/CAD, and has been disregarded. Please try again.")
        print("\n")
        
for i in assets:
    asset_str += i
    asset_str += " "

asset_str = asset_str[:-1]
print("\n Your selected tickers are: ", asset_str)

assets.sort()

What is the ticker of your desired stock?: shop
What is the ticker of your desired stock?: f
What is the ticker of your desired stock?: tsla
What is the ticker of your desired stock?: amzn
What is the ticker of your desired stock?: m

 Your selected tickers are:  SHOP F TSLA AMZN M


In [5]:
startdate = '2022-04-01'
enddate = date ## spans from the beginning of 2022-Q2 to yesterday's date from input
CAD2USD = yf.Ticker("CADUSD=x")
exr = (CAD2USD.history(start=startdate, end=enddate))[['Close']]
exr.index = exr.index.strftime('%Y-%m-%d')

def data(tickers, start, end):
    data = yf.download(tickers, start=start, end=end)['Adj Close']
    data.index = data.index.strftime('%Y-%m-%d')
    return data 
    
def conversion(data):
    for ticker in assets:
        if yf.Ticker(ticker).info.get("currency").upper() == "USD":
            rates = data[ticker] / exr['Close']
            rates = rates.dropna()
            data[ticker] = rates
    return data

def pct_data(data):
    return data.pct_change().dropna() * 100

def pct_wrapper(tickers, start, end):
    return pct_data(conversion(data(tickers, start, end)))  

In [6]:
returns_global = pct_wrapper(asset_str, startdate, enddate)
rfrr = 0 # risk-free return rate; we'll let this be 0

def neg_sharpe(distribution):
    portf_return = np.sum(returns_global.mean() * distribution) * trading_days # calculating return
    
    covar = returns_global.cov() * trading_days
    variance = np.dot(distribution.T, np.dot(covar, distribution))
    portf_volatility = np.sqrt(variance) # volatility -- represents the stdev
    
    return - ((portf_return - rfrr) / portf_volatility) # negative sharpe ratio to then use minimization optim.

[*********************100%%**********************]  5 of 5 completed


In [7]:
def max_sharpe():
    initial_distrib = num_assets * [1/num_assets]
    
    # Define the constraint function
    def constraint_func(distrib):
        return np.sum(distrib) - 1
    
    # Define the bounds
    bounds = [(0, 1) for _ in range(num_assets)]
    
    # Call minimize with both constraint and bounds
    allocation = minimize(neg_sharpe, initial_distrib, constraints={'type': 'eq', 'fun': constraint_func}, bounds=bounds, method='SLSQP')
    
    return allocation.x

optimal = max_sharpe()

In [8]:
print("The raw optimal investment distribution is as follows:")
optimaldf = pd.DataFrame({'Allocation': optimal}, index=assets)
optimaldf

The raw optimal investment distribution is as follows:


,Allocation
AMZN,8.027504e-01
F,3.522900e-16
M,1.424583e-01
SHOP,5.479123e-02
TSLA,0.000000e+00


In [9]:
print("In most scenarios, this will be hard to decipher. Here are the simplified allocations:")
print("Note: The total may be 1 less or above 100% due to rounding errors")
acc = 0

while acc < len(optimal):
    optimal[acc] = np.round(optimal[acc] * 100)
    acc += 1

smp_optimaldf = pd.DataFrame({"Simplified Allocation": optimal}, index=assets)
smp_optimaldf


In most scenarios, this will be hard to decipher. Here are the simplified allocations:
Note: The total may be 1 less or above 100% due to rounding errors


,Simplified Allocation
AMZN,80.0
F,0.0
M,14.0
SHOP,5.0
TSLA,0.0
